<h1> Q2: Gradient Descent and Logistic Regression </h1>

In [4]:
import os
import autograd.numpy as np  # when testing gradient
from cvxopt import matrix, solvers
import pandas as pd

print("Testing")

Testing


In [5]:
#Q2a)


def linearRegL2Obj(w, X, y):

    pred = X @ w
    diff = pred = y
    objectiveVal = (1/2) * np.mean(diff ** 2)
    n = X.shape[0]
    gradient = (X.T @ diff) / n

    return objectiveVal, gradient


In [6]:
#Q2b

def gd(func, w_init, X, y, step_size, max_iter, tol=1e-10):

    w = w_init

    for i in range(max_iter):
        objval, gradient = func(w,X,y)
        
        norm = np.linalg.norm(gradient)
        if norm < tol: #stop when gradient is good enough
            break;

        w = w - step_size * gradient

    return w



In [7]:
#Q2c

def sigmoid(x):
    return np.exp(-np.logaddexp(0, -x))


def logisticRegObj(w, X, y):
    pred = X @ w
    n = X.shape[0]
    
    objval = (1/n) * np.sum(np.logaddexp(0, -pred) - y * pred)
    grad = (1/n) * X.T @ (sigmoid(pred) - y)
    
    return objval, grad


In [8]:
#q2d
def synClsExperiments():

    def genData(n_points, d):
        '''
        This function generate synthetic data
        '''
        c0 = np.ones([1, d])  # class 0 center
        c1 = -np.ones([1, d])  # class 1 center
        X0 = np.random.randn(n_points, d) + c0  # class 0 input
        X1 = np.random.randn(n_points, d) + c1  # class 1 input
        X = np.concatenate((X0, X1), axis=0)
        X = np.concatenate((np.ones((2 * n_points, 1)), X), axis=1)  # augmentation
        y = np.concatenate([np.zeros([n_points, 1]), np.ones([n_points, 1])], axis=0)
        return X, y

    def runClsExp(m=100, d=2, eta=0.1, max_iter=1000, tol=1e-10):
        '''
        Run classification experiment with the specified arguments
        '''

        Xtrain, ytrain = genData(m, d)
        n_test = 1000
        Xtest, ytest = genData(n_test, d)

        w_init = np.random.randn(d + 1, 1)
        w_logit = gd(logisticRegObj, w_init, Xtrain, ytrain, eta, max_iter, tol)
        ytrain_hat = 0.5 * (1 + np.sign(Xtrain @ w_logit))
        train_acc = np.sum(ytrain == ytrain_hat) / (2 * m)

        ytest_hat = 0.5 * (1 + np.sign(Xtest @ w_logit))
        test_acc = np.sum(ytest == ytest_hat) / (2 * n_test)

        return train_acc, test_acc

    n_runs = 100
    train_acc = np.zeros([n_runs, 4, 3])
    test_acc = np.zeros([n_runs, 4, 3])
    for r in range(n_runs):
        for i, m in enumerate((10, 50, 100, 200)):
            train_acc[r, i, 0], test_acc[r, i, 0] = runClsExp(m=m)
        for i, d in enumerate((1, 2, 4, 8)):
            train_acc[r, i, 1], test_acc[r, i, 1] = runClsExp(d=d)
        for i, eta in enumerate((0.1, 1.0, 10., 100.)):
            train_acc[r, i, 2], test_acc[r, i, 2] = runClsExp(eta=eta)

    return np.mean(train_acc, axis=0), np.mean(test_acc, axis=0)



synClsExperiments()


(array([[0.9465  , 0.8416  , 0.922   ],
        [0.9264  , 0.9218  , 0.92435 ],
        [0.9261  , 0.9811  , 0.9315  ],
        [0.919425, 0.9994  , 0.90155 ]]),
 array([[0.903235, 0.840325, 0.91979 ],
        [0.91746 , 0.919805, 0.919335],
        [0.9196  , 0.97506 , 0.91998 ],
        [0.920155, 0.995805, 0.896885]]))